### 문제
- 수직으로 정렬된 베리어 3개에 구멍의 x좌표가 각각 있고, 베리어의 가장 위에서 아래로 직선을 통과시킨다고 했을 때 직선을 통과시키는 경우의 개수를 세는 문제.

### 접근
- 맨 위의 베리어부터 a, b, c라고 하자.
- a -> b으로의 증가량과 b -> c로의 증가량이 같아야 한다. 그렇게 증가하는 경우의 수를 센다.
  - (힌트) $a - b = b - c$ 인 경우의 수를 찾는 것이다. 식을 정리하면 $a + c = 2b$ 라는 식이 나온다.
- 우선 a + c에 대한 다항식의 곱셈을 시도해볼 수 있다.
- 어떠한 결과가 나왔을때 그 차수를 2로 나눈 값이 b에 존재하면 계수만큼 곱한 다음 더해주는 것이 답이 아닐까?
  - 이 경우 convolution을 한 뒤의 차수가 홀수일 경우에 답이 될 수 없는데, 직접 써보면 자명하다는 것을 관찰할 수 있다.
    - 예를들어 a가 2, c가 5인 경우에, 정수인 b가 존재할 수 없다.
    

In [ ]:
import io, os, math
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

p = 469762049
def ntt(a, inv=False):
  n = len(a)
  j = 0
  for i in range(1,n):
    rev = n >> 1
    while j >= rev:
      j -= rev
      rev >>= 1
    j += rev
    if i < j:
      a[i], a[j] = a[j], a[i]

  step = 2
  while step <= n:
    half = step // 2
    u = pow(3,p//step,p)
    if inv: u = pow(u,p-2,p)
    for i in range(0, n, step):
      w = 1
      for j in range(i, i + half):
        v = a[j + half] * w
        a[j + half] = (a[j] - v)% p
        a[j] += v
        a[j] %= p
        w *= u
        w %= p
    step *= 2

  if inv:
    invn = p - (p-1) // n
    for i in range(n):
      a[i] = (a[i] * invn)%p
  return a
    
def ntt_conv(a, b):
  s = len(a) + len(b) - 1
  n = 1 << s.bit_length()
  a += [0 for _ in range(n - len(a))]
  b += [0 for _ in range(n - len(b))]
  ntt(a)
  ntt(b)
  for i in range(n):
    a[i] *= b[i]
  ntt(a,True)
  return a

MAX = 30000
SIZE = math.ceil(math.log2(MAX * 2))
def sol() :
  _1 = input()
  A = [*map(int, input().split())]
  _2 = input()
  B = [*map(int, input().split())]
  _3 = input()
  C = [*map(int, input().split())]

  P = [0] * (1 << (SIZE + 1))
  Q = [0] * (1 << (SIZE + 1))
  for a in A :
    P[a + MAX] = 1
  
  for c in C:
    Q[c + MAX] = 1
  
  R = ntt_conv(P, Q)
  ans = 0
  for v in B :
    x = R[(v + MAX) * 2]
    ans += x
  
  print(ans)

sol()

- a+c=2b 수식을 유도하는 부분의 힌트만 얻었다. 거의 다왔는데 아쉽다.
- ntt를 사용하는 구현으로 바꿨다.
- P와 Q를 업데이트 하는 과정에서 zip(A, C)를 사용하다가 WA를 한번 받았다. 수열의 길이가 다를 수 있음을 항상 명심하자.